### SparkSession

In [17]:
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('join') \
    .getOrCreate()

### JOIN

#### The first JOIN

In [ ]:
df_green_revenue = spark.read.parquet('data/report/revenue/green/')

In [ ]:
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow/')

In [ ]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [ ]:
# keep all records from both DataFrames
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.write.parquet('data/report/revenue/total/')

#### The second JOIN

In [19]:
df_join = spark.read.parquet('data/report/revenue/total/')

In [ ]:
df_zones = spark.read.option("header", "true").csv('../module1/docker_sql/data/taxi_zone_lookup.csv')

In [ ]:
# filter records that "zone" column is not match to "LocationID" column
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [43]:
df_result.drop('LocationID', 'zone').write.parquet('data/tmp/revenue-zones')

In [44]:
df_result.drop('LocationID', 'zone').show(20)

+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|               hour|green_amount|green_number_records|yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|2020-01-01 01:00:00|       15.29|                   1|         null|                 null|Manhattan|   Boro Zone|
|2020-01-01 02:00:00|        null|                null|        15.96|                    1|Manhattan| Yellow Zone|
|2020-01-01 02:00:00|        null|                null|         11.8|                    1|Manhattan| Yellow Zone|
|2020-01-01 02:00:00|        null|                null|        36.09|                    2|Manhattan| Yellow Zone|
|2020-01-01 03:00:00|        null|                null|         16.3|                    1|Manhattan| Yellow Zone|
|2020-01-01 06:00:00|        null|                null|         17.8|           